In [ ]:
import qililab as ql
from functools import partial
from fluqe.transpilers.single_qubit_2level import Qubit2LevelTranspiler
from fluqe.functions.qubit import epsilon_two_level, persistent_current_sim, delta_sim

### Constants
Here we define the runcard path, the calibration path, and some dummy annealing program.
We also define a transpiler model and load an instance of it.

In [ ]:
RUNCARD_PATH = "runcards/galadriel.yml"
CALIBRATION_PATH = "runcards/calibration_data.yml"

annealing_program_dict = [{
        "qubit_0": {"sigma_x": 0, "sigma_y": 0, "sigma_z": 1},
        "qubit_1": {"sigma_x": 0.1, "sigma_y": 0.1, "sigma_z": 0.1},
        "coupler_0_1": {"sigma_x": 1, "sigma_y": 0.2, "sigma_z": 0.2},
    },
    {
        "qubit_0": {"sigma_x": 0.1, "sigma_y": 0.1, "sigma_z": 1.1},
        "qubit_1": {"sigma_x": 0.2, "sigma_y": 0.2, "sigma_z": 0.2},
        "coupler_0_1": {"sigma_x": 0.9, "sigma_y": 0.1, "sigma_z": 0.1},
    },
    {
        "qubit_0": {"sigma_x": 0.3, "sigma_y": 0.3, "sigma_z": 0.7},
        "qubit_1": {"sigma_x": 0.5, "sigma_y": 0.2, "sigma_z": 0.01},
        "coupler_0_1": {"sigma_x": 0.5, "sigma_y": 0, "sigma_z": -1},
    },
]

eps_partial = partial(epsilon_two_level, persistent_current=persistent_current_sim)
transpiler = Qubit2LevelTranspiler(epsilon_model=eps_partial, delta_model=delta_sim)

### Loading the platform and calibration object

In [ ]:
platform = ql.build_platform(runcard=RUNCARD_PATH)
loaded_calibration = ql.Calibration.load_from(CALIBRATION_PATH)

### Executing the annealing schedule using the calibrated measurement and the transpiler.

In [ ]:
results = platform.execute_anneal_program(
    annealing_program_dict=annealing_program_dict,
    transpiler=transpiler,
    averages=2,
    readout_bus="readout_bus",
    measurement_name="readout",
    weights="optimal_weights",
    calibration=CALIBRATION_PATH,
)